<a href="https://colab.research.google.com/github/chloepark627/calculating-distance-data-science-intro-000/blob/master/ch07_RNN(7_4_NLG).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Natural Language Generation 자연어생성(p.216~)

In [0]:
import tensorflow as tf
import numpy as np

In [0]:
path_to_file = tf.keras.utils.get_file('input_.txt', 'http://bit.ly/2Mc3SOV')

In [0]:
train_text = open(path_to_file, 'rb').read().decode(encoding='utf-8')

In [21]:
len(train_text)

26265493

In [22]:
train_text[:100]

'\ufeff태조 이성계 선대의 가계. 목조 이안사가 전주에서 삼척·의주를 거쳐 알동에 정착하다 \n태조 강헌 지인 계운 성문 신무 대왕(太祖康獻至仁啓運聖文神武大王)의 성은 이씨(李氏)요, 휘'

In [23]:
print(train_text[:100])

﻿태조 이성계 선대의 가계. 목조 이안사가 전주에서 삼척·의주를 거쳐 알동에 정착하다 
태조 강헌 지인 계운 성문 신무 대왕(太祖康獻至仁啓運聖文神武大王)의 성은 이씨(李氏)요, 휘


In [0]:
# remove stopwords

import re
# https://github.com/yoonkim/CNN_sentence/blob/master/process_data.py -> 추가 필요

def clean_str(string, TREC=False):
    """
    Tokenization/string cleaning for all datasets except for SST.
    Every dataset is lower cased except for TREC
    """
    string = re.sub(r"[^가-힣A-Za-z0-9(),!?\'\`]", " ", string)     
    string = re.sub(r"\'s", " \'s", string) 
    string = re.sub(r"\'ve", " \'ve", string) 
    string = re.sub(r"n\'t", " n\'t", string) 
    string = re.sub(r"\'re", " \'re", string) 
    string = re.sub(r"\'d", " \'d", string) 
    string = re.sub(r"\'ll", " \'ll", string) 
    string = re.sub(r",", " , ", string) 
    string = re.sub(r"!", " ! ", string) 
    string = re.sub(r"\(", " \( ", string) 
    string = re.sub(r"\)", " \) ", string) 
    string = re.sub(r"\?", " \? ", string) 
    string = re.sub(r"\s{2,}", " ", string)    
    string = re.sub(r"\'{2,}", "\'", string)    
    string = re.sub(r"\'", "", string)    
    return string.strip().lower()

In [25]:
print(train_text[:50])
# 개행구분으로 문장나누고, stopwords제거 
train_text = train_text.split('\n')
train_text = [clean_str(sentence) for sentence in train_text]
train_text_X = []

print(train_text[:50])
# 문장 내 탭구분으로 단어나누고, 문장끝에 개행 추가
for sentence in train_text:
    train_text_X.extend(sentence.split())
    train_text_X.append('\n')
print(train_text_X[:50])
print(len(train_text_X))

# 빈 문자 제거
train_text_X = [word for word in train_text_X if word != '']
print(train_text_X[:50])
print(len(train_text_X))

﻿태조 이성계 선대의 가계. 목조 이안사가 전주에서 삼척·의주를 거쳐 알동에 정착하다 
태조 강헌 지인 계운 성문 신무 대왕(太祖康獻至仁啓運聖文
['태조 이성계 선대의 가계 목조 이안사가 전주에서 삼척 의주를 거쳐 알동에 정착하다', '태조 강헌 지인 계운 성문 신무 대왕 \\( \\) 의 성은 이씨 \\( \\) 요 , 휘 \\( \\) 는 단 \\( \\) 이요 , 자 \\( \\) 는 군진 \\( \\) 이다 그전의 휘 \\( \\) 는 이성계 \\( \\) 요 , 호 \\( \\) 는 송헌 \\( \\) 이다 전주 \\( \\) 의 대성 \\( \\) 이다 사공 \\( \\) 휘 \\( \\) 이한 \\( \\) 이 신라 \\( \\) 에 벼슬하여 태종왕 \\( \\) 001 \\) 의 10대 \\( \\) 손자인 군윤 \\( \\) 김은의 \\( \\) 의 딸에게 장가들어 시중 \\( \\) 휘 \\( \\) 이자연 \\( \\) 을 낳았다 시중이 복야 \\( \\) 휘 \\( \\) 이천상 \\( \\) 을 낳고 , 복야가 아간 \\( \\) 휘 \\( \\) 광희 \\( \\) 를 낳고 , 아간이 사도 \\( \\) 삼중 대광 \\( \\) 휘 \\( \\) 입전 \\( \\) 을 낳고 , 사도가 휘 \\( \\) 이긍휴 \\( \\) 를 낳고 , 이긍휴가 휘 \\( \\) 염순 \\( \\) 을 낳고 , 염순이 휘 \\( \\) 이승삭 \\( \\) 을 낳고 , 이승삭이 휘 \\( \\) 충경 \\( \\) 을 낳고 , 충경이 휘 \\( \\) 경영 \\( \\) 을 낳고 , 경영이 휘 \\( \\) 충민 \\( \\) 을 낳고 , 충민이 휘 \\( \\) 화 \\( \\) 를 낳고 , 화가 휘 \\( \\) 진유 \\( \\) 를 낳고 , 진유가 휘 \\( \\) 궁진 \\( \\) 을 낳고 , 궁진이 대장군 \\( \\) 휘 \\( \\) 용부 \\( \\) 를 낳고 , 대장군이 내시 집주 \\( \\) 002 \\) 휘 \\( \\) 이인 

In [32]:
vocab = sorted(set(train_text_X))
vocab.append('UNK')
print('{} unique words.'.format(len(vocab)))
print('[vocab] ', vocab[:10])

word2idx = {w:i for i,w in enumerate(vocab)}
idx2word = np.array(vocab)

332515 unique words.
[vocab]  ['\n', '!', ',', '000명으로', '001', '002', '003', '004', '005', '006']


In [37]:
text_as_int = np.array([word2idx[word] for word in train_text_X])

for word,_ in zip(word2idx, range(10)):
    print('{} : {}, '.format(repr(word), word2idx[word]))

print(word2idx['UNK']) # of 332,515 unique words

'\n' : 0, 
'!' : 1, 
',' : 2, 
'000명으로' : 3, 
'001' : 4, 
'002' : 5, 
'003' : 6, 
'004' : 7, 
'005' : 8, 
'006' : 9, 
332514


In [38]:
print(train_text_X[:20])
print(text_as_int[:20])

['태조', '이성계', '선대의', '가계', '목조', '이안사가', '전주에서', '삼척', '의주를', '거쳐', '알동에', '정착하다', '\n', '태조', '강헌', '지인', '계운', '성문', '신무', '대왕']
[299181 229520 161349  17366 110944 230178 250960 154993 225348  28960
 190192 256005      0 299181  25557 273428  36069 163902 180371  84330]


In [0]:
# tf.data.Dataset 으로 학습을 위한 데이터세트 생성하기

# 장점: 데이터 섞기, 배치수만큼 자르기, 다른 데이터세트에 매핑하기 등을 빠르게 수행

In [45]:
seq_length = 25 
examples_per_epoch = len(text_as_int) // seq_length
print(seq_length, examples_per_epoch)

#25개 단어가 주어지면, 다음 단어를 예측!
sentence_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

# 전체 데이터를 (25+1)개 단어 단위로 쪼갠 후, 나머지를 한 덩어리로 남긴다(True)
sentence_dataset = sentence_dataset.batch(seq_length+1, drop_remainder=True) 

for item in sentences_dataset.take(1):
    print(idx2word[item.numpy()])
    print(item.numpy())

25 348081
태조
299181
